In [1]:
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.validation import check_is_fitted
from sklearn.exceptions import NotFittedError
from urllib.request import urlopen 
from pyspark import SparkContext 
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext 
plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)

In [2]:
spark = SparkSession.builder.master("local[2]").appName("app1").getOrCreate()

In [3]:
train_data_df = pd.read_csv("/Users/shubhambhardwaj/Desktop/Data_Science/Techgig/bank_fraud/application_train.csv")

Code for cverting data frame (pandas) to spark dataframe

spark.conf.set("spark.sql.execution.arrow.enabled", "true")
train_data = spark.createDataFrame(train_data_df)

In [55]:
# Code for directly reading a csv
train_data = spark.read.option("header","true").option("inferschema", "true").csv("/Users/shubhambhardwaj/Desktop/Data_Science/Techgig/bank_fraud/application_train.csv")

In [5]:
train_data.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [6]:
train_data.count()

257512

In [7]:
train_data.select("SK_ID_CURR").show(2)

+----------+
|SK_ID_CURR|
+----------+
|    157876|
|    157878|
+----------+
only showing top 2 rows



Converting the data to table 

In [8]:
train_data.registerTempTable('train_table')
sqlContext =  SQLContext(spark.sparkContext)

In [9]:
sqlContext.sql('select SK_ID_CURR,FLAG_DOCUMENT_21 from train_table').show(5)

+----------+----------------+
|SK_ID_CURR|FLAG_DOCUMENT_21|
+----------+----------------+
|    157876|               0|
|    157878|               0|
|    157879|               0|
|    157880|               0|
|    157881|               0|
+----------+----------------+
only showing top 5 rows



In [10]:
train_data.rdd.getNumPartitions()

2

In [11]:
train_data.rdd.repartition(10)

MapPartitionsRDD[31] at coalesce at NativeMethodAccessorImpl.java:0

In [12]:
train_data.select("SK_ID_CURR").show(2)

+----------+
|SK_ID_CURR|
+----------+
|    157876|
|    157878|
+----------+
only showing top 2 rows



In [13]:
train_data_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,157876,0,Cash loans,F,N,Y,0,67500.0,343800.0,16155.0,225000.0,Unaccompanied,State servant,Secondary / secondary special,Married,House / apartment,0.015221,-19421,-826,-293.0,-2651,NaN,1,1,1,1,1,0,Medicine staff,2.0,2,2,TUESDAY,14,0,0,0,1,1,0,Medicine,NaN,0.583251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,4.0,1.0,-180,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,157878,0,Cash loans,M,Y,N,2,247500.0,945000.0,40167.0,945000.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.019101,-15322,365243,-7733.0,-4788,6.0,1,0,0,1,0,0,NaN,4.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,XNA,0.342195,0.623227,0.622922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,-1965,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
2,157879,0,Revolving loans,M,N,Y,2,180000.0,540000.0,27000.0,540000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.006207,-11120,-61,-953.0,-3474,NaN,1,1,0,1,1,0,Sales staff,4.0,2,2,SATURDAY,13,0,1,1,0,1,1,Business Entity Type 3,0.076905,0.430050,0.656158,0.0976,NaN,0.9861,NaN,NaN,0.1064,0.0917,0.3333,NaN,0.0577,NaN,0.0964,NaN,0.0304,0.0693,NaN,0.9851,NaN,NaN,0.1208,0.1034,0.3333,NaN,0.0568,NaN,0.0704,NaN,0.0322,0.1124,NaN,0.9861,NaN,NaN,0.12,0.1034,0.3333,NaN,0.0587,NaN,0.1123,NaN,0.031,NaN,NaN,0.0531,"Stone, brick",No,0.0,0.0,0.0,0.0,-1585,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
3,157880,0,Cash loans,F,N,Y,0,112500.0,295168.5,16011.0,238500.0,Una

In [14]:
sqlContext.sql('select distinct OCCUPATION_TYPE from train_table').show()

+--------------------+
|     OCCUPATION_TYPE|
+--------------------+
|                null|
|            Managers|
|            HR staff|
|      Medicine staff|
|         Accountants|
|            Laborers|
|      Cleaning staff|
|Private service s...|
|             Drivers|
|         Sales staff|
|       Realty agents|
|            IT staff|
|      Security staff|
|         Secretaries|
|  Low-skill Laborers|
|          Core staff|
|       Cooking staff|
|High skill tech s...|
|Waiters/barmen staff|
+--------------------+



In [15]:
print("Here's the dimensions of our data frame:\n", 
     train_data_df.shape)
print("Here's the data types of our columns:\n",
     train_data_df.dtypes)

Here's the dimensions of our data frame:
 (257512, 122)
Here's the data types of our columns:
 SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int64
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int64
FLAG_EMP_PHONE                 

In [16]:
#train_data_df.apply(lambda x: (x.isnull().values.ravel().sum())  )

Fetching only columns where null values are present

In [50]:
null_columns=train_data_df.columns[train_data_df.isnull().any()]
train_data_df[null_columns].isnull().sum()

AMT_ANNUITY                         11
AMT_GOODS_PRICE                    240
NAME_TYPE_SUITE                   1100
OCCUPATION_TYPE                  80737
CNT_FAM_MEMBERS                      1
EXT_SOURCE_2                       534
EXT_SOURCE_3                     51021
YEARS_BEGINEXPLUATATION_AVG     125613
FLOORSMAX_AVG                   128145
YEARS_BEGINEXPLUATATION_MODE    125613
FLOORSMAX_MODE                  128145
YEARS_BEGINEXPLUATATION_MEDI    125613
FLOORSMAX_MEDI                  128145
TOTALAREA_MODE                  124283
EMERGENCYSTATE_MODE             122057
OBS_30_CNT_SOCIAL_CIRCLE           853
DEF_30_CNT_SOCIAL_CIRCLE           853
OBS_60_CNT_SOCIAL_CIRCLE           853
DEF_60_CNT_SOCIAL_CIRCLE           853
AMT_REQ_CREDIT_BUREAU_HOUR       34785
AMT_REQ_CREDIT_BUREAU_DAY        34785
AMT_REQ_CREDIT_BUREAU_WEEK       34785
AMT_REQ_CREDIT_BUREAU_MON        34785
AMT_REQ_CREDIT_BUREAU_QRT        34785
AMT_REQ_CREDIT_BUREAU_YEAR       34785
dtype: int64

Function to get the columns having null values greater then any value passed as a parameter in the function

In [18]:
def null_columns(num):
    null_columns_gt_257510=train_data_df.columns[train_data_df.isnull().sum()>num]
    print(train_data_df[null_columns_gt_257510].isnull().sum())

In [19]:
null_columns(170000)

YEARS_BUILD_AVG             171249
COMMONAREA_AVG              179905
FLOORSMIN_AVG               174748
LIVINGAPARTMENTS_AVG        175973
NONLIVINGAPARTMENTS_AVG     178800
YEARS_BUILD_MODE            171249
COMMONAREA_MODE             179905
FLOORSMIN_MODE              174748
LIVINGAPARTMENTS_MODE       175973
NONLIVINGAPARTMENTS_MODE    178800
YEARS_BUILD_MEDI            171249
COMMONAREA_MEDI             179905
FLOORSMIN_MEDI              174748
LIVINGAPARTMENTS_MEDI       175973
NONLIVINGAPARTMENTS_MEDI    178800
FONDKAPREMONT_MODE          176104
dtype: int64


Function to get the columns having percentage of null values greater then any value passed as a parameter in the function

In [20]:
def null_columns_percent(num):
    null_columns_percentage=train_data_df.columns[train_data_df.isnull().mean().round(4) * 100 > float(num)]
    print(train_data_df[null_columns_percentage].isnull().mean().round(4) * 100)
    return null_columns_percentage


In [21]:
null_col_toBeDropped= null_columns_percent(50)

OWN_CAR_AGE                 66.01
EXT_SOURCE_1                56.39
APARTMENTS_AVG              50.75
BASEMENTAREA_AVG            58.54
YEARS_BUILD_AVG             66.50
COMMONAREA_AVG              69.86
ELEVATORS_AVG               53.29
ENTRANCES_AVG               50.34
FLOORSMIN_AVG               67.86
LANDAREA_AVG                59.36
LIVINGAPARTMENTS_AVG        68.34
LIVINGAREA_AVG              50.18
NONLIVINGAPARTMENTS_AVG     69.43
NONLIVINGAREA_AVG           55.19
APARTMENTS_MODE             50.75
BASEMENTAREA_MODE           58.54
YEARS_BUILD_MODE            66.50
COMMONAREA_MODE             69.86
ELEVATORS_MODE              53.29
ENTRANCES_MODE              50.34
FLOORSMIN_MODE              67.86
LANDAREA_MODE               59.36
LIVINGAPARTMENTS_MODE       68.34
LIVINGAREA_MODE             50.18
NONLIVINGAPARTMENTS_MODE    69.43
NONLIVINGAREA_MODE          55.19
APARTMENTS_MEDI             50.75
BASEMENTAREA_MEDI           58.54
YEARS_BUILD_MEDI            66.50
COMMONAREA_MED

In [22]:
print("Here's the dimensions of our data frame:\n", 
     train_data_df.shape)
print("Here's the data types of our columns:\n",
     train_data_df.dtypes)

Here's the dimensions of our data frame:
 (257512, 122)
Here's the data types of our columns:
 SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int64
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int64
FLAG_EMP_PHONE                 

In [23]:
def print_dx_perc(data_frame, col):
    """Function used to print class distribution for our data set"""
    try:
        # Stores value counts
        col_vals = data_frame[col].value_counts()
        # Resets index to make index a column in data frame
        col_vals = col_vals.reset_index()
        # If the number of unique instances in column exceeds 20 print warning
        if len(col_vals['index']) > 20:
            print('Warning: values in column are more than 20 \nPlease try a column with lower value counts!')
        # Else it calculates/prints percentage for each unique value in column
        else:
            # Create a function to output the percentage
            f = lambda x, y: 100 * (x / sum(y))
            for i in range(0, len(col_vals['index'])):
                print('{0} accounts for {1:.2f}% of the {2} column'\
                      .format(col_vals['index'][i],
                              f(col_vals[col].iloc[i],
                                col_vals[col]),
                              col))
    # try-except block goes here if it can't find the column in data frame
    except KeyError as e:
        print('{0}: Not found'.format(e))
        print('Please choose the right column name!')

In [24]:
print_dx_perc(train_data_df, 'TARGET')

0 accounts for 91.92% of the TARGET column
1 accounts for 8.08% of the TARGET column


In [25]:
sqlContext.sql('select count(*)  from train_table where TARGET<1').show()

+--------+
|count(1)|
+--------+
|  236713|
+--------+



In [60]:
sqlContext.sql('select count(*)  from train_table where TARGET>0').show()

+--------+
|count(1)|
+--------+
|   20799|
+--------+



In [63]:
sqlContext.sql('select count(*)  from train_table where SK_ID_CURR is not null ').show()

+--------+
|count(1)|
+--------+
|  257512|
+--------+



In [59]:
from pyspark.sql.functions import isnan, when, count, col

train_data.select([count(when(isnan(c), c)).alias(c) for c in train_data.columns]).show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------+------------+---------------------------+-------------+----------------------------+--------------+----------------------------+--------------+--------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+--------------------------+-------------------------+-------

In [28]:
null_col_toBeDropped

Index(['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
       'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
       'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG',
       'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE',
       'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
       'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
       'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
       'WALLSMATERIAL_MODE'],
      dtype='object')

In [29]:
#Removing null columns with more than 60%null values
train_data_df.drop( null_col_toBeDropped, inplace=True, axis=1)

In [56]:
train_data = train_data.drop(*null_col_toBeDropped)

In [31]:
#dropping some more columns afer analysing that they have least effect on target values

In [32]:
more_col_toBeDropped=['FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',  'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']    

In [33]:
train_data_df.drop(more_col_toBeDropped,inplace=True, axis=1)

In [57]:
train_data = train_data.drop(*more_col_toBeDropped)

In [35]:
train_data_df.shape

(257512, 55)

In [36]:

train_data_df.dtypes

SK_ID_CURR                        int64
TARGET                            int64
NAME_CONTRACT_TYPE               object
CODE_GENDER                      object
FLAG_OWN_CAR                     object
FLAG_OWN_REALTY                  object
CNT_CHILDREN                      int64
AMT_INCOME_TOTAL                float64
AMT_CREDIT                      float64
AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
NAME_TYPE_SUITE                  object
NAME_INCOME_TYPE                 object
NAME_EDUCATION_TYPE              object
NAME_FAMILY_STATUS               object
NAME_HOUSING_TYPE                object
REGION_POPULATION_RELATIVE      float64
DAYS_BIRTH                        int64
DAYS_EMPLOYED                     int64
DAYS_REGISTRATION               float64
DAYS_ID_PUBLISH                   int64
OCCUPATION_TYPE                  object
CNT_FAM_MEMBERS                 float64
REGION_RATING_CLIENT              int64
REGION_RATING_CLIENT_W_CITY       int64


In [37]:
train_data.dtypes

[('SK_ID_CURR', 'int'),
 ('TARGET', 'int'),
 ('NAME_CONTRACT_TYPE', 'string'),
 ('CODE_GENDER', 'string'),
 ('FLAG_OWN_CAR', 'string'),
 ('FLAG_OWN_REALTY', 'string'),
 ('CNT_CHILDREN', 'int'),
 ('AMT_INCOME_TOTAL', 'double'),
 ('AMT_CREDIT', 'double'),
 ('AMT_ANNUITY', 'double'),
 ('AMT_GOODS_PRICE', 'double'),
 ('NAME_TYPE_SUITE', 'string'),
 ('NAME_INCOME_TYPE', 'string'),
 ('NAME_EDUCATION_TYPE', 'string'),
 ('NAME_FAMILY_STATUS', 'string'),
 ('NAME_HOUSING_TYPE', 'string'),
 ('REGION_POPULATION_RELATIVE', 'double'),
 ('DAYS_BIRTH', 'int'),
 ('DAYS_EMPLOYED', 'int'),
 ('DAYS_REGISTRATION', 'double'),
 ('DAYS_ID_PUBLISH', 'int'),
 ('OCCUPATION_TYPE', 'string'),
 ('CNT_FAM_MEMBERS', 'int'),
 ('REGION_RATING_CLIENT', 'int'),
 ('REGION_RATING_CLIENT_W_CITY', 'int'),
 ('WEEKDAY_APPR_PROCESS_START', 'string'),
 ('HOUR_APPR_PROCESS_START', 'int'),
 ('REG_REGION_NOT_LIVE_REGION', 'int'),
 ('REG_REGION_NOT_WORK_REGION', 'int'),
 ('LIVE_REGION_NOT_WORK_REGION', 'int'),
 ('REG_CITY_NOT_LIVE_C

In [38]:
int_col=''
double_col=''
str_col=''
for name,dtype in train_data.dtypes:
    if dtype =='int':
        int_col = int_col+'\''+name+'\','
    if dtype == 'double':
        double_col = double_col+'\''+name+'\','
    if dtype == 'string':
        str_col = str_col+'\''+name+'\','
        
            

In [39]:
int_col

"'SK_ID_CURR','TARGET','CNT_CHILDREN','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_ID_PUBLISH','CNT_FAM_MEMBERS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','HOUR_APPR_PROCESS_START','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR',"

In [40]:
double_col

"'AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','EXT_SOURCE_2','EXT_SOURCE_3','YEARS_BEGINEXPLUATATION_AVG','FLOORSMAX_AVG','YEARS_BEGINEXPLUATATION_MODE','FLOORSMAX_MODE','YEARS_BEGINEXPLUATATION_MEDI','FLOORSMAX_MEDI','TOTALAREA_MODE',"

In [41]:
str_col

"'NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','EMERGENCYSTATE_MODE',"

In [42]:
train_data = train_data.drop(*['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','EXT_SOURCE_2','EXT_SOURCE_3','YEARS_BEGINEXPLUATATION_AVG','FLOORSMAX_AVG','YEARS_BEGINEXPLUATATION_MODE','FLOORSMAX_MODE','YEARS_BEGINEXPLUATATION_MEDI','FLOORSMAX_MEDI','TOTALAREA_MODE'])

In [58]:
train_data.dropna()

DataFrame[SK_ID_CURR: int, TARGET: int, NAME_CONTRACT_TYPE: string, CODE_GENDER: string, FLAG_OWN_CAR: string, FLAG_OWN_REALTY: string, CNT_CHILDREN: int, AMT_INCOME_TOTAL: double, AMT_CREDIT: double, AMT_ANNUITY: double, AMT_GOODS_PRICE: double, NAME_TYPE_SUITE: string, NAME_INCOME_TYPE: string, NAME_EDUCATION_TYPE: string, NAME_FAMILY_STATUS: string, NAME_HOUSING_TYPE: string, REGION_POPULATION_RELATIVE: double, DAYS_BIRTH: int, DAYS_EMPLOYED: int, DAYS_REGISTRATION: double, DAYS_ID_PUBLISH: int, OCCUPATION_TYPE: string, CNT_FAM_MEMBERS: int, REGION_RATING_CLIENT: int, REGION_RATING_CLIENT_W_CITY: int, WEEKDAY_APPR_PROCESS_START: string, HOUR_APPR_PROCESS_START: int, REG_REGION_NOT_LIVE_REGION: int, REG_REGION_NOT_WORK_REGION: int, LIVE_REGION_NOT_WORK_REGION: int, REG_CITY_NOT_LIVE_CITY: int, REG_CITY_NOT_WORK_CITY: int, LIVE_CITY_NOT_WORK_CITY: int, ORGANIZATION_TYPE: string, EXT_SOURCE_2: double, EXT_SOURCE_3: double, YEARS_BEGINEXPLUATATION_AVG: double, FLOORSMAX_AVG: double, YEA

Preparing Data for Machine Learning
The process includes Category Indexing, One-Hot Encoding and VectorAssembler — a feature transformer that merges multiple columns into a vector column.

In [64]:
cols = train_data.columns

In [65]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','EMERGENCYSTATE_MODE']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'TARGET', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['SK_ID_CURR','TARGET','CNT_CHILDREN','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_ID_PUBLISH','CNT_FAM_MEMBERS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','HOUR_APPR_PROCESS_START','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [66]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(train_data)
train_data = pipelineModel.transform(train_data)
selectedCols = ['label', 'features'] + cols
train_data = train_data.select(selectedCols)
train_data.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nu

In [67]:
train, test = train_data.randomSplit([0.7, 0.3],seed=2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Py4JJavaError: An error occurred while calling o4290.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 94.0 failed 1 times, most recent failure: Lost task 1.0 in stage 94.0 (TID 354, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.sort_addToSo

In [68]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train_data)

Py4JJavaError: An error occurred while calling o4338.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 96.0 failed 1 times, most recent failure: Lost task 1.0 in stage 96.0 (TID 356, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:221)
	at org.apache.spark.stora